<a href="https://colab.research.google.com/github/malak128/cloud/blob/main/cv%20project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub --quiet

import kagglehub
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [ ]:
import os

import kagglehub
path = kagglehub.dataset_download("moltean/fruits")
print("Dataset path:", path)

# Show first few items inside
for root, dirs, files in os.walk(path):
    print(root)
    print(dirs)
    break


Dataset path: /kaggle/input/fruits
/kaggle/input/fruits
['fruits-360_100x100', 'fruits-360_3-body-problem', 'fruits-360_dataset_meta', 'fruits-360_original-size', 'fruits-360_multi']


In [ ]:
path = kagglehub.dataset_download("moltean/fruits")
print("Dataset path:", path)

train_set = "/kaggle/input/fruits/fruits-360_100x100/fruits-360/Training"
test_set  = "/kaggle/input/fruits/fruits-360_100x100/fruits-360/Test"



Dataset path: /kaggle/input/fruits


In [ ]:
image_size = (128, 128)
batch_size = 64

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_set,
    image_size=image_size,
    batch_size=batch_size
)
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_set,
    image_size=image_size,
    batch_size=batch_size
)

Found 104456 files belonging to 207 classes.
Found 34864 files belonging to 207 classes.


In [ ]:
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
test_dataset = test_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))


In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(207, activation='softmax')
])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5
)


Epoch 1/5
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 264s 154ms/step - accuracy: 0.0581 - loss: 4.6939 - val_accuracy: 0.2366 - val_loss: 3.2908
Epoch 2/5
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 112s 68ms/step - accuracy: 0.3207 - loss: 2.8674 - val_accuracy: 0.4179 - val_loss: 2.4861
Epoch 3/5
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 111s 68ms/step - accuracy: 0.5045 - loss: 2.0833 - val_accuracy: 0.5150 - val_loss: 2.0354
Epoch 4/5
1483/1633 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.6065 - loss: 1.6368

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_finetuning = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5
)

Epoch 1/5
1633/1633 ━━━━━━━━━━━━━━━━━━━━ 164s 90ms/step - accuracy: 0.5361 - loss: 13.6222 - val_accuracy: 0.8093 - val_loss: 0.8380
Epoch 2/5
 553/1633 ━━━━━━━━━━━━━━━━━━━━ 1:12 68ms/step - accuracy: 0.9182 - loss: 0.2839

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"✅ Test Accuracy: {acc:.4f}")
print(f"✅ Test Loss: {loss:.4f}")


In [ ]:
plt.plot(history.history['accuracy'] + history_finetuning.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'] + history_finetuning.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
